##### Sensitivity Analysis

with variation of :
- Controlled Energy Producer Maximum output 
- Total Bt Production (global BT energy producer in the higher level network) i.e. the added power is shared propotionally among all LV producer in the lower Network



Simulation based on  : 
* Prediction block RNN trained in [RNN_Train_StLaurentDeJourdes](RNN_Train_StLaurentDeJourdes.ipynb) 
    * To predict $X(k)$ and $Y(k)$  as respectively  $\tilde{X(k)}$ an  $\tilde{Y(k)}$ (Predictions on testing set are done in [RNN_Sim_StLaurentDeJourdes](RNN_Sim_StLaurentDeJourdes.ipynb))
* Prediction Block Future Known [2021_2022_KnownFuture](2021_2022_KnownFuture.ipynb)
* Persistence Model [2021_2022_Persistence](2021_2022_Persistence.ipynb)


Maximum voltage rise $vm_{pu}^{max} = 1.0250 $

---

In [1]:
def block_pf_Opf(par_engines, 
                 pred_model_f=None
                ):
    """
Execute a power flow or an optimal power flow depending on the opf_status that is extracted from one of the parallel 
engine object and return a dataFrame. If the retrieved opf_status is 'Both', the function is used as the block PF/OPF
and thereforth the returned dataframe columns consist of ['max_vm_pu_pf', 'max_vm_pu', hvProd_N, ...,  hvProd_Z] where 
max_vm_pu_pf ==> max voltage  before opf, max_vm_pu==> maximum voltage after opf , hvProd_N ==> Name of first higher 
voltage  Producer (Egg: P0013) in the Lower Network and hvProd_Z ==>  Name of last HV  Producer (Egg: P0100)  in  the  
Lower Network. If opf_status is False, the function is used as the block PF, the returned dataframe columns consist  
only of max_vm_pu_pf the maximum voltage rise observed on the network after the control is implemented.
  
Parameters
----------
par_engines: (oriCs.CreateParEngines(n_engines))
    Parallel engines object. 
pred_model_f: (str) optional, default to None
    'Pers' ==> Using persistence model

    """
    
    
    
    # Note that all the variables used in the parallel running are already sent to the local space of each engine
    if pred_model_f == 'Pers': 
    # Run problem in parallel
        %px par_run_Results = [par_oriFunctions.run_powerflow_at(lowerNet, cur_period+1, lowerNet_hv_activated_bus, uppNet_sum_max_lvProdLoad,  dict_df_sgenLoad, vm_mu_max, opf_status, pred_model) for cur_period in period_part]
    else:
        %px par_run_Results = [par_oriFunctions.run_powerflow_at(lowerNet, cur_period, lowerNet_hv_activated_bus, uppNet_sum_max_lvProdLoad,  dict_df_sgenLoad, vm_mu_max, opf_status) for cur_period in period_part]
        
    # Gather the results of all the engines in a unique variable
    results = par_engines.gather_results('par_run_Results')
    
    # Wait 2seconds time for gathering the results of parralel computing  
    time.sleep(2) 

    # return depends on opf_status defined in parallel engines
    if par_engines.get_opf_status() == 'Both': 
        extracted_results = par_engines.get_results_asDf() # Extract results
        return extracted_results
    
    elif par_engines.get_opf_status() == False: 
        # Put data in dataframe
        max_vm_pu_rnn_df = pd.DataFrame(data=np.array(results), 
                                        index=par_engines.get_run_periodIndex(), columns=['vm_pu_max_pf'],)
        return max_vm_pu_rnn_df
    else: pass


#### Import Modules 


In [2]:
import pandas as pd
import pandapower as pp
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm # Profiling 
import seaborn as sbn
import pickle, sys, importlib,  time
import os
from pickle import load
import joblib

In [3]:
function_folder = 'py_files/' 
# Add function folder to path if it is not already
if function_folder not in sys.path: sys.path.append(function_folder)

import oriFunctions as oriFc
import oriClass as oriCs
import checker
from oriVariables import network_folder, excel_folder, py_folder, Δt, attr_list, testSet_end_date

#### Import Networks


In [4]:
importlib.reload(oriCs) 
importlib.reload(oriFc)

<module 'oriFunctions' from 'py_files\\oriFunctions.py'>

#### Import Cleaned data files for component of the network

In [5]:
df_data = joblib.load(network_folder+'CleanedData.pkl')


# Extract only dailight period i.e. from 07am to 7PM
# The daylight period is considered to be defined betwenn 07am and 7Pm excluded. 
h_start_end = ('07:00','18:50') # for the persistance model, the previous period i.e. 06:50 
                                # is needed to compute the first instant i.e. 07:00
per_index = df_data.index
per_daylight = ( pd.Series(index=per_index.to_timestamp(), dtype=object).between_time(*h_start_end) ).index.to_period('10T')
day_tot_per = len(per_daylight[(per_daylight.year==2020)&(per_daylight.month==1)&(per_daylight.day==1)])


# Put all the data in a unique dataframe
df_data = df_data.loc[per_daylight]

# # Extract only the relavant testing set since the training set covers the first part of the data
df_final = df_data[(df_data.index>='2021 06 01') & (df_data.index<=testSet_end_date)]
per_index = df_final.index
per_index2 = ( pd.Series(index=per_index.to_timestamp(), dtype=object
                           ).between_time('07:00','18:50') ).index.to_period('10T')

df_final_copy = df_final.copy(deep=True)

# Check wether the input dataframe columns are in the expected order
checker.check_networkDataDf_columnsOrder(df_final)

## OPF  RNN

#### Set parameters on the Lower Network

#### Start Parallel engines

In [6]:
n_engines = os.cpu_count() - 1  # Define total number of parallel engines to use. 
# Make sure it is lower than the total number of cores in the computer otherwise 
# no ressources is available to work when the simulation is running. 
parEngines = oriCs.CreateParEngines(n_engines)
                   

Starting 7 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/7 [00:00<?, ?engine/s]

---

#### Define Parameters for Network

In [7]:
# Define  networks Files
lowerNet_file = 'CIVAUX.p'
upperNet_file = 'ST LAURENT.p'

# Define How the added BT production will be share among the Lover voltage producers. 
# See setNetwork_params(*args ) documentation for more information
coef_add_bt_dist = 'lowNet'
ctrld_hvProdName = 'P0100'

# Extract all the HV producer 
hvProdNames = {elm for elm in df_final.columns if elm[-4:].isdecimal()}
non_ctrld_hvProdNames_list = list(hvProdNames.difference({ctrld_hvProdName}))
vm_mu_max, vm_mu_min = 1.0250, 0.95  # Add maximum voltage constraints on the Buses in the network 
dailyPeriod_interval = ('07:10','18:50')

#### Define parameters to use to save simulation results

In [8]:
ctrld_hvProd_max = 4  #  Define the maximum power output subscribed by the controlled HV  producer
 
bt_add_max = 4       # Define Maximum power for the added LV production

# Define Range for the maximum power output subscribed by P0100 
ctrld_hvProd_max_range = np.arange(0, ctrld_hvProd_max+0.1 ,0.2)
bt_add_range =  np.arange(0, bt_add_max+0.1,.2);

simNames = []
for bt_elm in bt_add_range: 
    for elm in ctrld_hvProd_max_range: 
        simNames.append(f'BT+={np.round(bt_elm, decimals=2)} {ctrld_hvProdName}={np.round(elm, decimals=2)}')
        
simNames = np.array(simNames).reshape((len(bt_add_range),-1))

### Persistence model

In [10]:
ctrld_hvProdName

'P0100'

In [ ]:
n_start, n_end = 0 , 1
# n_end = len(bt_add_range)
for bt_elm_index, coef_add_bt in enumerate(bt_add_range[n_start:n_end]):
    
    # Set Network
    networks = oriFc.setNetwork_params(upperNet_file, lowerNet_file, ctrld_hvProdName,
                                       (coef_add_bt, coef_add_bt_dist), 
                                       (vm_mu_max, vm_mu_min))
    
    pred_model = 'Pers'
    # Create dictionnary that will be sent to the parallel engines
    lowerNet_hv_activated_bus = networks.get_lowerNet_hvActivatedBuses(networks.get_lowerNet_hv_bus_df().index)
    parameters_dict = dict({'lowerNet':networks.get_lowerNet(), 
                            'lowerNet_hv_activated_bus':lowerNet_hv_activated_bus, 
                            'uppNet_sum_max_lvProdLoad':networks.get_upperNet_sum_max_lvProdLoad(), 
                            'vm_mu_max':vm_mu_max,
                            'pred_model':pred_model
                           })

    sim_res_Dict = {}  # Dictionnary to save variables
    bt_elm_index = bt_elm_index+n_start

    
    # for mr_index in range(len(ctrld_hvProd_max_range)):
    for mr_index in [20]:
        
        ctrld_hvUpscaling_coef = ctrld_hvProd_max_range[mr_index]  # Define Higher voltage upsacaling coeff

        # Create dictionnary of main variable i.e. Prod BT, HT, Cons to used in parallel engines 
        dict_df_sgenLoad = oriFc.createDict_prodHtBt_Load(df_final,
                                                          networks, 
                                                          ctrld_hvUpscaling_coef,
                                                          ctrld_hvProd_max
                                                         )
        
        print(f'*******************  Simulation:{bt_elm_index+1}/{len(bt_add_range) } ==> {mr_index+1}/{len(ctrld_hvProd_max_range)}  ***********')
        print("                           ------- Bloc PF/OPF ---------         ")
        df_prodHT = dict_df_sgenLoad['df_prodHT']
        ##          ---------------------------  BLOCK PF/OPF  ------------------------         ##
        # Initialize the parallel engine parameters i.e. send varible into each local space
        opf_status = 'Both'
        parEngines.sendVar_to_localSpace(df_prodHT.index, opf_status, dict_df_sgenLoad, parameters_dict, 0 )

        # cALL Block pf/op
        pf_opf_block_result = block_pf_Opf(parEngines, pred_model)    

#         # ---------------------- Implement Robust Persistence ------------------------------------
        oriFc.robustPersistence(pf_opf_block_result,df_final[[ctrld_hvProdName]],
                                ctrld_hvUpscaling_coef, ctrld_hvProd_max, vm_mu_max)

#         ##             ---------------------------  BLOCK PROD  ------------------------         ##
        oriFc.block_prod(pf_opf_block_result, df_final[[ctrld_hvProdName]],
                         ctrld_hvUpscaling_coef, ctrld_hvProd_max)

        
        # get periods in order
        per_index2 = (pf_opf_block_result.index.to_timestamp().to_series(
                        ).between_time(*dailyPeriod_interval).index.to_period('10T') )

        # Update Values for the dict to use for final power flow bloc
        day_totPer = day_tot_per-1
        hvProd_df = pd.concat([df_final.loc[per_index2[day_totPer:], non_ctrld_hvProdNames_list], 
                            pf_opf_block_result.loc[per_index2[day_totPer:], [ctrld_hvProdName]]], axis=1)

        dict_df_sgenLoad = dict({'df_prodHT': hvProd_df, 
                                 'df_prod_bt_total':df_final.loc[per_index2[day_totPer:], ['Prod_BT']], 
                                 'df_cons_total':df_final.loc[per_index2[day_totPer:], ['Cons']] } )


        ##             ---------------------------  BLOCK PF  ------------------------         ##
        print("                            ------- Bloc PF ---------         ")

        opf_status = False
        # Initialize the parallel engine parameters i.e. send varible into each local space for PF
        parEngines.sendVar_to_localSpace(hvProd_df.index, opf_status, dict_df_sgenLoad, parameters_dict, 1)

        # cALL Block pf
        max_vm_pu_pers_df = block_pf_Opf(parEngines)    

        #####                             Saving Results                         -------------------------------
        ctrld_hvProd_noControl_df = df_final.loc[hvProd_df.index, [ctrld_hvProdName]]
        ctrld_hvProd_noControl_upscaled_df = ctrld_hvProd_max_range[mr_index]*ctrld_hvProd_noControl_df/ctrld_hvProd_max
        ctrld_hvProd_pers_df = pd.DataFrame(data=pf_opf_block_result.loc[hvProd_df.index][ctrld_hvProdName].values, 
                                     index=pf_opf_block_result.loc[hvProd_df.index].index, 
                                     columns=['Pers_Rob Sim'+str(mr_index+1)] )
        # Concat
        ctrld_hvProd_concat_df = pd.concat([ctrld_hvProd_noControl_upscaled_df,
                                     ctrld_hvProd_pers_df], axis=1)
        # Rename
        ctrld_hvProd_concat_df.rename(columns=({ctrld_hvProdName:'No_Control'}), inplace=True);

        int_sim_res = {'maxV_rise_df':max_vm_pu_pers_df,              # Intermediate results
                       'Power Sgen': ctrld_hvProd_concat_df}   
        
        sim_res_Dict.update([(simNames[bt_elm_index, mr_index],int_sim_res)])   # Final results
        
    file_name= 'Pers_btRangeIndex_' +str(bt_elm_index)+ '.plk'
    folder_name = 'pickle_files/simulationResults/SensAnalysis/Pers2/'
    joblib.dump(sim_res_Dict,folder_name+file_name)

In [17]:
ctrld_hvProd_noControl_df.columns[0]

'P0100'

#### FUTURE KNOWN

In [86]:
ctrld_hvProd_df = df_final_copy[[ctrld_hvProdName]]
n_start, n_end = 0, -1

for bt_elm_index, coef_add_bt in enumerate(bt_add_range[n_start:]):
    # Set Network
    networks = oriFc.setNetwork_params(upperNet_file, lowerNet_file, ctrld_hvProdName,
                                       (coef_add_bt, coef_add_bt_dist), 
                                       (vm_mu_max, vm_mu_min))
    
    # Create dictionnary that will be sent to the parallel engines
    lowerNet_hv_activated_bus = networks.get_lowerNet_hvActivatedBuses(networks.get_lowerNet_hv_bus_df().index)
    parameters_dict = dict({'lowerNet':networks.get_lowerNet(), 
                            'lowerNet_hv_activated_bus':lowerNet_hv_activated_bus, 
                            'uppNet_sum_max_lvProdLoad':networks.get_upperNet_sum_max_lvProdLoad(), 
                            'vm_mu_max':vm_mu_max,
                           })

    sim_res_Dict = {}  # Dictionnary to save variables
    bt_elm_index = bt_elm_index+n_start

    for mr_index in range(len(ctrld_hvProd_max_range)):
        ctrld_hvUpscaling_coef = ctrld_hvProd_max_range[mr_index]  # Define Higher voltage upsacaling coeff

        # Create dictionnary of main variable i.e. Prod BT, HT, Cons to used in parallel engines 
        dict_df_sgenLoad = oriFc.createDict_prodHtBt_Load(df_final,
                                                          networks, 
                                                          ctrld_hvUpscaling_coef,
                                                          ctrld_hvProd_max
                                                         )

        print(f'***************************  Simulation:{bt_elm_index+1}/{len(bt_add_range) } ==> {mr_index+1}/{len(ctrld_hvProd_max_range)}      ***********')
        print("                           ------- Bloc PF/OPF ---------         ")
        ##          ---------------------------  BLOCK PF/OPF  ------------------------         ##
        # Initialize the parallel engine parameters i.e. send varible into each local space
        opf_status = 'Both'
        parEngines.sendVar_to_localSpace(df_prodHT.index, opf_status, dict_df_sgenLoad, parameters_dict, 0 )

        # cALL Block pf/op
        pf_opf_block_result = block_pf_Opf(parEngines)    

        
        # Get maximum voltage over the network 
        max_vm_pu_df_known = pf_opf_block_result[['max_vm_pu']]
        max_vm_pu_df_known.rename(columns={'max_vm_pu':'Fut_Known'}, inplace=True)
        max_vm_pu_pf_df = pf_opf_block_result[['max_vm_pu_pf']] 
        

        #####   Saving Results 
        mask = per_index2[day_tot_per-1:]
        P0100_known_df = pd.DataFrame(data=pf_opf_block_result.loc[mask][ctrd_hvProdName].values, 
                                     index=pf_opf_block_result.loc[mask].index, 
                                     columns=['Fut_Known Sim'+str(mr_index+1)] )
        
        
        ctrld_hvProd_concat_df = pd.concat([ctrld_hvProd_max_range[mr_index]*ctrld_hvProd_df.loc[mask]/ctrld_hvProd_max,
                                     P0100_known_df], axis=1)
        # Rename
        ctrld_hvProd_concat_df.rename(columns=({ctrld_hvProdName:'No_Control'}), inplace=True);


        int_sim_res = {'maxV_rise_df':max_vm_pu_df_known,              # Intermediate results
                       'Power Sgen': ctrld_hvProd_concat_df}   
        
        sim_res_Dict.update([ (simNames[bt_elm_index, mr_index], int_sim_res) ])   # Final results
        
    file_name= 'Fut_Known_btRangeIndex_' +str(bt_elm_index)+ '.plk'
    folder_name = 'pickle_files/simulationResults/SensAnalysis/FutureKnown2/'
    joblib.dump(sim_res_Dict,folder_name+file_name)

***************************  Simulation:1/16 ==> 1/21      ***********
                           ------- Bloc PF/OPF ---------         
importing numpy on engine(s)
importing pandapower on engine(s)
importing pandas on engine(s)
importing Par_myFunctions on engine(s)


%px:   0%|          | 0/48 [00:00<?, ?tasks/s]

***************************  Simulation:1/16 ==> 2/21      ***********
                           ------- Bloc PF/OPF ---------         
importing numpy on engine(s)
importing pandapower on engine(s)
importing pandas on engine(s)
importing Par_myFunctions on engine(s)


%px:   0%|          | 0/48 [00:00<?, ?tasks/s]

***************************  Simulation:1/16 ==> 3/21      ***********
                           ------- Bloc PF/OPF ---------         
importing numpy on engine(s)
importing pandapower on engine(s)
importing pandas on engine(s)
importing Par_myFunctions on engine(s)


...

%px:   0%|          | 0/48 [00:00<?, ?tasks/s]

***************************  Simulation:16/16 ==> 20/21      ***********
                           ------- Bloc PF/OPF ---------         
importing numpy on engine(s)
importing pandapower on engine(s)
importing pandas on engine(s)
importing Par_myFunctions on engine(s)


%px:   0%|          | 0/48 [00:00<?, ?tasks/s]

***************************  Simulation:16/16 ==> 21/21      ***********
                           ------- Bloc PF/OPF ---------         
importing numpy on engine(s)
importing pandapower on engine(s)
importing pandas on engine(s)
importing Par_myFunctions on engine(s)


%px:   0%|          | 0/48 [00:00<?, ?tasks/s]

---

#### USING ROBUST RNN 

##### Load prediction file

In [22]:
rnnPred_df = joblib.load('pickle_files/simulationResults/RNN_pred.pkl')

# Rename the collumn of the saved df before sending to  simulation
rnnPred_df.columns = df_final.columns

In [14]:
n_start, n_end = 0,- 1
# n_end = len(bt_add_range)
for bt_elm_index, coef_add_bt in enumerate(bt_add_range[n_start:]):
    # Set Network
    networks = oriFc.setNetwork_params(upperNet_file, lowerNet_file, ctrld_hvProdName,
                                       (coef_add_bt, coef_add_bt_dist), 
                                       (vm_mu_max, vm_mu_min))
    
    # Create dictionnary that will be sent to the parallel engines
    lowerNet_hv_activated_bus = networks.get_lowerNet_hvActivatedBuses(networks.get_lowerNet_hv_bus_df().index)
    parameters_dict = dict({'lowerNet':networks.get_lowerNet(), 
                            'lowerNet_hv_activated_bus':lowerNet_hv_activated_bus, 
                            'uppNet_sum_max_lvProdLoad':networks.get_upperNet_sum_max_lvProdLoad(), 
                            'vm_mu_max':vm_mu_max,
                           })
    
    sim_res_rnn_Dict = {}  # Dictionnary to save variables
    bt_elm_index = bt_elm_index+n_start

    for mr_index in range(len(ctrld_hvProd_max_range)): 
        ctrld_hvUpscaling_coef = ctrld_hvProd_max_range[mr_index]  # Define upsacaling coeff
       # Create dictionnary of main variable i.e. Prod BT, HT, Cons to used in parallel engines 
        dict_df_sgenLoad = oriFc.createDict_prodHtBt_Load(rnnPred_df,
                                                          networks, 
                                                          ctrld_hvUpscaling_coef,
                                                          ctrld_hvProd_max
                                                         )

        print(f'***************************  Simulation:{bt_elm_index+1}/{len(bt_add_range) } ==> {mr_index+1}/{len(ctrld_hvProd_max_range)}      ***********')
        print("                           ------- Bloc PF/OPF ---------         ")
        df_prodHT = dict_df_sgenLoad['df_prodHT']
        
        ##          ---------------------------  BLOCK PF/OPF  ------------------------         ##
        # Initialize the parallel engine parameters i.e. send varible into each local space
        opf_status = 'Both'
        parEngines.sendVar_to_localSpace(df_prodHT.index, opf_status, dict_df_sgenLoad, parameters_dict, 0 )

        # cALL Block pf/op
        pf_opf_block_result = block_pf_Opf(parEngines)    

        # ----------------------   Implement Robusteness   ------------------------------------
        # TODO: Change function robustPersistence() name in persistence()
        oriFc.robustPersistence(pf_opf_block_result, df_final[[ctrld_hvProdName]],
                                ctrld_hvUpscaling_coef, ctrld_hvProd_max, vm_mu_max)
        
       ##             ---------------------------  BLOCK PROD  ------------------------         ##
        oriFc.block_prod(pf_opf_block_result, df_final[[ctrld_hvProdName]],ctrld_hvUpscaling_coef, ctrld_hvProd_max)

        # get periods in order
        per_index2 = (pf_opf_block_result.index.to_timestamp().to_series(
                                        ).between_time(*dailyPeriod_interval).index.to_period('10T')
)

        # Update Values for the dict to use for final power flow bloc
        day_totPer = day_tot_per-1
        hvProd_df = pd.concat([df_final.loc[per_index2[day_totPer:], non_ctrld_hvProdNames_list],
                                pf_opf_block_result.loc[per_index2[day_totPer:], [ctrld_hvProdName]]], axis=1)

        dict_df_sgenLoad = dict({'df_prodHT': hvProd_df, 
                                 'df_prod_bt_total':df_final.loc[per_index2[day_totPer:], ['Prod_BT']], 
                                 'df_cons_total':df_final.loc[per_index2[day_totPer:], ['Cons']] } )


        ##             ---------------------------  BLOCK PF  ------------------------         ##
        print("                            ------- Bloc PF/OPF ---------         ")

        opf_status = False
        # Initialize the parallel engine parameters i.e. send varible into each local space for PF
        parEngines.sendVar_to_localSpace(hvProd_df.index, opf_status, dict_df_sgenLoad, parameters_dict, 1)

        # cALL Block pf
        max_vm_pu_rnn_df = block_pf_Opf(parEngines)     


        #####   Saving Results 
        ctrld_hvProd_noControl_df = df_final.loc[hvProd_df.index, [ctrld_hvProdName]]
        ctrld_hvProd_noControl_upscaled_df = ctrld_hvProd_max_range[mr_index]*ctrld_hvProd_noControl_df/ctrld_hvProd_max
        P0100_rnn_df = pd.DataFrame(data=pf_opf_block_result.loc[hvProd_df.index][ctrd_hvProdName].values, 
                                     index=pf_opf_block_result.loc[hvProd_df.index].index, 
                                     columns=['RNN Sim'+str(mr_index+1)] )
        # Concat
        ctrld_hvProd_concat_df = pd.concat([ctrld_hvProd_noControl_upscaled_df,
                                     P0100_rnn_df], axis=1)
        # Rename
        ctrld_hvProd_concat_df.rename(columns=({ctrld_hvProdName:'No_Control'}), inplace=True);


        int_sim_res = {'maxV_rise_df':max_vm_pu_rnn_df,              # Intermediate results
                       'Power Sgen': ctrld_hvProd_concat_df}         
        sim_res_rnn_Dict.update([(simNames[bt_elm_index, mr_index],int_sim_res)])   # Final results

    file_name= 'RNN_btRangeIndex_' +str(bt_elm_index)+ '.plk'
    folder_name = 'pickle_files/simulationResults/SensAnalysis/RNNRob/'
    joblib.dump(sim_res_rnn_Dict,folder_name+file_name)

***************************  Simulation:21/21 ==> 1/21      ***********
                           ------- Bloc PF/OPF ---------         
importing numpy on engine(s)
importing pandapower on engine(s)
importing pandas on engine(s)
importing Par_myFunctions on engine(s)


%px:   0%|          | 0/7 [00:00<?, ?tasks/s]

                            ------- Bloc PF/OPF ---------         


%px:   0%|          | 0/7 [00:00<?, ?tasks/s]

***************************  Simulation:21/21 ==> 2/21      ***********
                           ------- Bloc PF/OPF ---------         
importing numpy on engine(s)
importing pandapower on engine(s)
importing pandas on engine(s)
importing Par_myFunctions on engine(s)


...

%px:   0%|          | 0/7 [00:00<?, ?tasks/s]

***************************  Simulation:21/21 ==> 21/21      ***********
                           ------- Bloc PF/OPF ---------         
importing numpy on engine(s)
importing pandapower on engine(s)
importing pandas on engine(s)
importing Par_myFunctions on engine(s)


%px:   0%|          | 0/7 [00:00<?, ?tasks/s]

                            ------- Bloc PF/OPF ---------         


%px:   0%|          | 0/7 [00:00<?, ?tasks/s]